INSTALL
- FAISS - 
- Sent2Vec

In [1]:
import numpy as np
import faiss   

In [2]:
import os
import time
name = "Faiss-EM-OPUS-bert-en-vi"+ str(time.strftime("%Y%m%d-%H%M"))

!mkdir $name
path = os.getcwd() + "/" + name
os.chdir(path)

!pwd
os.chdir(path)

/home/chinh/clc_data/Script/Faiss-EM-OPUS-bert-en-vi20210304-1327


In [3]:
!wget https://github.com/hoangtrungchinh/clc_data/raw/master/opus-100-corpus.zip
!mkdir data_bin
!unzip 'opus-100-corpus.zip' -d 'data_bin'

--2021-03-04 13:27:05--  https://github.com/hoangtrungchinh/clc_data/raw/master/opus-100-corpus.zip
Resolving github.com (github.com)... 52.74.223.119
Connecting to github.com (github.com)|52.74.223.119|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/opus-100-corpus.zip [following]
--2021-03-04 13:27:06--  https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/opus-100-corpus.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30690851 (29M) [application/zip]
Saving to: ‘opus-100-corpus.zip’

opus-100-corpus.zip 100%[===================>]  29,27M  5,47MB/s    in 5,1s    

2021-03-04 13:27:14 (5,73 MB/s) - ‘opus-100-corpus.zip’ saved [30690851/30690851]

In [4]:
# import file
import numpy as np
import glob

en_files = sorted(glob.glob("data_bin/*.en"))
vi_files = sorted(glob.glob("data_bin/*.vi"))
print(en_files)
print(vi_files)

lst_en = []
lst_vi = []

start = time.time()

for file in en_files:
  with open(file) as file_in:
    for line in file_in:
      lst_en.append(line.strip())

for file in vi_files:
  with open(file) as file_in:
    for line in file_in:
      lst_vi.append(line.strip())

print(len(lst_en))
print(len(lst_vi))

['data_bin/opus.en-vi-dev.en', 'data_bin/opus.en-vi-test.en', 'data_bin/opus.en-vi-train.en']
['data_bin/opus.en-vi-dev.vi', 'data_bin/opus.en-vi-test.vi', 'data_bin/opus.en-vi-train.vi']
1004000
1004000


In [5]:
# SPLIT TRAIN, TEST, VALID
from sklearn.model_selection import train_test_split
en_train, en_test_valid, vi_train, vi_test_valid =  train_test_split(lst_en, lst_vi, test_size=0.2, random_state=123)
en_valid, en_test, vi_valid, vi_test = train_test_split(en_test_valid, vi_test_valid, test_size=0.5, random_state=123)
print(len(en_train))
print(len(vi_train))
print(len(en_valid))
print(len(vi_valid))
print(len(en_test))
print(len(vi_test))
print(en_train[20])
print(vi_train[20])
print(en_valid[-1])
print(vi_valid[-1])
print(en_test[-1])
print(vi_test[-1])

803200
803200
100400
100400
100400
100400
Childs?
Childs?
It's powered up now, isn't it?
Bây giờ nó bật rồi phải không nhỉ?
Why the hell are you going to Cuddy's baby naming?
Bị làm sao mà anh lại đi dự lễ đặt tên em bé của Cuddy chứ?


In [6]:
file_variable = [en_train,vi_train,en_valid,vi_valid,en_test,vi_test]
file_names = ["en_train","vi_train","en_valid","vi_valid","en_test","vi_test"]

for i in range(len(file_names)):
  with open(file_names[i], 'w') as f:
    for item in file_variable[i]:
      f.write(item)
      f.write("\n")
    
!ls -al

total 105172
drwxrwxr-x 3 chinh chinh     4096 Thg 3   4 13:27 .
drwxrwxr-x 6 chinh chinh     4096 Thg 3   4 13:27 ..
drwxrwxr-x 2 chinh chinh     4096 Thg 3   4 13:27 data_bin
-rw-rw-r-- 1 chinh chinh  3318349 Thg 3   4 13:27 en_test
-rw-rw-r-- 1 chinh chinh 26563375 Thg 3   4 13:27 en_train
-rw-rw-r-- 1 chinh chinh  3328557 Thg 3   4 13:27 en_valid
-rw-rw-r-- 1 chinh chinh 30690851 Thg 3   4 13:27 opus-100-corpus.zip
-rw-rw-r-- 1 chinh chinh  4365722 Thg 3   4 13:27 vi_test
-rw-rw-r-- 1 chinh chinh 35019161 Thg 3   4 13:27 vi_train
-rw-rw-r-- 1 chinh chinh  4382771 Thg 3   4 13:27 vi_valid


In [7]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('stsb-bert-large')
# # model = SentenceTransformer('LaBSE')
# #Sentences are encoded by calling model.encode()
# emb1 = model.encode("This is a red cat with a hat.")
# emb2 = model.encode("Have you seen my red cat?")

# cos_sim = util.pytorch_cos_sim(emb1, emb2)
# print("Cosine-Similarity:", cos_sim)

In [8]:
# print(cos_sim.numpy()[0][0])

In [9]:
start = time.time()
srcfile = "en_train"
tarfile = "vi_train"
lst_srcfile = en_train
lst_tarfile = vi_train

vec_lst_srcfile = model.encode(lst_srcfile)
print('=== Ending, Total time (second): ', time.time() - start)

=== Ending, Total time (second):  4250.3012664318085


Test Cosin similarity

In [10]:
import faiss
# print(vec_lst_srcfile.shape[1])
start = time.time()
index = faiss.IndexFlatL2(vec_lst_srcfile.shape[1])
index.ntotal
faiss.normalize_L2(vec_lst_srcfile)
index.add(vec_lst_srcfile)
k = 6
distance, index = index.search(vec_lst_srcfile, k)

print('=== Ending, Total time (second): ', time.time() - start)
print(len(distance))
print(len(index))

print(index)
print(distance)

=== Ending, Total time (second):  3419.6206061840057
803200
803200
[[ 76309 157897 178919      0 236179 233242]
 [     1 364452 144270 466705 178997   7736]
 [ 87166      2 665949 113807 653536 342496]
 ...
 [803197 680232 231625 193396 117066 726611]
 [527692 180813 803198 768526 620014 591275]
 [112858 803199 233037 547484 412678 603907]]
[[2.3841858e-07 2.3841858e-07 2.3841858e-07 2.3841858e-07 4.7683716e-07
  4.7683716e-07]
 [0.0000000e+00 7.5549066e-01 7.8083515e-01 7.8685856e-01 7.8685868e-01
  7.9649842e-01]
 [5.9604645e-07 7.1525574e-07 4.4238329e-02 4.4238329e-02 4.4238329e-02
  4.4238448e-02]
 ...
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.1877370e-01
  1.5424967e-01]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 1.6851425e-02 1.6851544e-02
  3.0599356e-02]
 [1.1920929e-07 2.3841858e-07 2.0524204e-01 2.4202156e-01 2.4600720e-01
  2.4600732e-01]]


In [11]:
j = k-1
# TEST
for i in range(10):
    a = vec_lst_srcfile[i]
    b = vec_lst_srcfile[index[i][j]]
    dist_squared = np.sum(np.square(a - b))
    print("index:",index[i][j], "distance:",distance[i][j], "tested:",dist_squared)

index: 233242 distance: 4.7683716e-07 tested: 9.880157e-13
index: 7736 distance: 0.7964984 tested: 0.79649854
index: 342496 distance: 0.04423845 tested: 0.04423824
index: 689250 distance: 0.13553846 tested: 0.13553843
index: 716971 distance: 0.3948524 tested: 0.39485264
index: 439779 distance: 0.27048755 tested: 0.27048755
index: 712179 distance: 0.30991995 tested: 0.30992
index: 396271 distance: 0.3393216 tested: 0.33932066
index: 146392 distance: 0.60712147 tested: 0.6071214
index: 717325 distance: 0.4028021 tested: 0.4028022


In [12]:
start = time.time()
saperate = " || "

src_label = "S"
emb_label = "E"
thres_arr = [0.9, 0.8, 0.7, 0.6]
for i in range(len(thres_arr)):
    threshold = thres_arr[i]
    out_file_write = open("en_train_EM_"+str(thres_arr[i]), 'w')
    out_file_write_score = open("en_train_EM_score_"+str(thres_arr[i]), 'w')
    out_file_write_factor = open("en_train_EM_factor_"+str(thres_arr[i]), 'w')
    num_gth_thres = 0

    for i in range(len(lst_srcfile)):
        score = distance[i][1]
        content = ""
        content_label_arr = []
        if score >= threshold:
            num_gth_thres += 1
            best_simi_index = index[i][1]
            content = lst_srcfile[i] + saperate + lst_tarfile[best_simi_index]

            content_label = [src_label] * len(lst_srcfile[i].split())
            content_label.append(emb_label)
            content_label = content_label + [emb_label] * len(lst_tarfile[best_simi_index].split())

            content_label_arr = " ".join(content_label)
        else:
            content = lst_srcfile[i]
            content_label = [src_label] * len(lst_srcfile[i].split())
            content_label_arr = " ".join(content_label)

        out_file_write.writelines(content+ "\n")    
        out_file_write_factor.writelines(content_label_arr + "\n")
        out_file_write_score.writelines(str(score) + "\n")

    print('threshold =', threshold, "EM sentences =", num_gth_thres, "/", len(lst_srcfile), num_gth_thres/len(lst_srcfile), "%" )

    out_file_write.close()
    out_file_write_score.close()
    out_file_write_factor.close()

done = time.time()
elapsed = done - start
print('=== Ending, Total time (second): ', elapsed)

threshold = 0.9 EM sentences = 7435 / 803200 0.009256723107569722 %
threshold = 0.8 EM sentences = 29080 / 803200 0.036205179282868524 %
threshold = 0.7 EM sentences = 72197 / 803200 0.08988670318725099 %
threshold = 0.6 EM sentences = 134867 / 803200 0.1679121015936255 %
=== Ending, Total time (second):  25.040567874908447
